# 深層学習ノートブック-10 GPUの利用
Google ColaboratoryのGPUを使ってtensorの計算を行ってみる。  
※以下コードはGoogle ColaboratoryのGPU: T4を使って実行すること。

* tensor.to()メソッドでtensorを特定のデバイスに移動させる(新しいtensorを作成)  
  * .to(‘cuda’)でGPUに移動させる。（CUDAで計算できるようにする。）
  * .to(‘cpu’)でCPUに移動させる (デフォルトではCPU上に作られる)
  * .to()で移動したtensorは別の新たなtensorとなることに注意


* torch.cuda.is_available()でGPUの有無を確認できる
  * GPUが使える状態であればTrueを返す
* 異なる演算デバイス（CPU,GPU）にあるtensor同士は演算できないことに注意。

In [2]:
import torch
import time

## ■ CPUとGPUの計算速度比較
巨大なtensorの演算を行う際のCPUとGPUの計算速度の違いを見てみる。

In [3]:
# GPU使用可能
torch.cuda.is_available()

True

In [5]:
# CPU上でtensorを作成
tensor_cpu = torch.randn(10000, 10000)

# GPU上でTensorを作成（もしGPUが利用可能ならば）
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_gpu = tensor_cpu.to(device) # GPU上に新たに作成

# CPU上での計算の時間を測定
start_time = time.time()
result_cpu = torch.mm(tensor_cpu, tensor_cpu)
end_time = time.time()
print(f"Time taken on CPU: {end_time - start_time:.5f} seconds")

# GPU上での計算の時間を測定（もしGPUが利用可能ならば）
if device == 'cuda':
  start_time = time.time()
  result_gpu = torch.mm(tensor_gpu, tensor_gpu)
  end_time = time.time()
  print(f"Time taken on GPU: {end_time - start_time:.5f} seconds")

Time taken on CPU: 27.71705 seconds
Time taken on GPU: 0.11138 seconds


上記のように行列の積演算において、GPUを使用した場合の方が圧倒的に早い。  
確かに積演算は並列化しやすそう。  
深層学習において、行列の積演算は頻繁に行うので、GPUによる計算は必須といえる。

## ■ MNISTのMLP学習ループをGPUで実行
学習でGPUを使用するためには全ての計算の元になる特徴量やパラメタのtensorについて、.to('cuda)しておく必要がある。  
なお、以下のような2層のMLPのような単純なモデルならCPUとGPUで計算速度はそれほど変わらない。

In [13]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [14]:
device = 'cuda'

# 隠れ層・出力層の入力側に相当するクラス
class Linear:
    def __init__(self, n_input, n_output) -> None:
        self.W = ( torch.randn((n_output, n_input)) * torch.sqrt(torch.tensor(2./n_input)) ).to(device) # GPUで計算するために.to('cuda')
        self.W.requires_grad = False
        self.B = torch.zeros(size=(1, n_output)).to(device) # GPUで計算するために.to('cuda')
        self.B.requires_grad = False # .to(device)した後はrequires_gradの設定が引き継がれないので、.to()した後にrequires_gradを設定する。

    def forward(self, X): # Xには特徴量および隠れ層のAが入る。
        self.X = X # backwardでも計算に使いたいので、インスタンス変数に格納
        self.Z = X @ self.W.T + self.B
        return self.Z

    def backward(self, Z):
        self.W.grad_ = Z.grad_.T @ self.X
        self.B.grad_ = torch.sum(Z.grad_, dim=0)
        self.X.grad_ = Z.grad_ @ self.W
        return self.X.grad_ # Xおよび隠れ層のAの勾配を返す


# 隠れ層の出力側に相当するクラス
class ReLU:
    def forward(self, Z):
        self.Z = Z
        return torch.where(Z > 0 , Z, 0.) # Aを返す

    def backward(self, A):
        self.Z.grad_ = A.grad_ * ( self.Z > 0 ).float()
        return self.Z.grad_


# 損失関数を計算するクラス
class SoftmaxCrossEntoropy:
    def forward(self, X, y_true):
        max_val = X.max(dim=1, keepdim=True).values
        # 各要素のe^xを計算（これが分子になる）
        e_x = (X - max_val).exp()
        denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
        self.softmax_out = e_x / denominator
        self.loss = - (y_true * torch.log(self.softmax_out + 1e-10)).sum() / y_true.shape[0]

        return self.loss, self.softmax_out

    def backward(self, y_true): # 損失から最終出力の勾配を計算する。
        return (self.softmax_out - y_true) / y_true.shape[0] #softmax_outはyの予測値(Z2およびA2に同じ)


In [15]:
# 隠れ層１層の場合のMLP（スクラッチ）
class MlpModel:

    # コンストラクタでは特徴量数、隠れ層のノード数、出力層のノード数を受け取る。
    def __init__(self, n_features, hidden_units, output_units) -> None:
        self.Linear_1 = Linear(n_features, hidden_units)
        self.relu = ReLU()
        self.Linear_2 = Linear(hidden_units, output_units)
        self.loss_func = SoftmaxCrossEntoropy()


    def forward(self, X, y):
        self.X = X
        self.Z1 = self.Linear_1.forward(X)
        self.A1 = self.relu.forward(self.Z1)
        self.Z2 = self.Linear_2.forward(self.A1)
        # 出力層の活性化関数は恒等関数としているので、loss_funcの引数はA2ではなくZ2でOK
        self.loss, y_pred = self.loss_func.forward(self.Z2, y)
        return self.loss , y_pred


    # 明にW,Bを計算していないが、メソッド内部で計算されてインスタンス変数として保持されていることに注意。
    def backward(self, y):
        self.Z2.grad_ = self.loss_func.backward(y) # lossが求まったということはひとつ前の出力Z2の勾配が求まる
        self.A1.grad_ = self.Linear_2.backward(self.Z2) # Z2の勾配が求まったということはひとつ前のA1の勾配が求まる。（同時にW2,B2の勾配も求まる）
        self.Z1.grad_ = self.relu.backward(self.A1) # A1の勾配が求まったということはひとつ前のZ1の勾配が求まる。
        self.X.grad_ = self.Linear_1.backward(self.Z1) # Z1の勾配が求まったということはひとつ前のXの勾配が求まる。（Xの勾配自体は格納しなくてもOk）


    # 勾配の初期化
    def zero_grad(self):
        self.Linear_1.W.grad_ = None
        self.Linear_1.B.grad_ = None
        self.Linear_2.W.grad_ = None
        self.Linear_2.B.grad_ = None


    # パラメタの更新
    def step(self, learning_rate):
        self.Linear_1.W -= learning_rate * self.Linear_1.W.grad_
        self.Linear_1.B -= learning_rate * self.Linear_1.B.grad_
        self.Linear_2.W -= learning_rate * self.Linear_2.W.grad_
        self.Linear_2.B -= learning_rate * self.Linear_2.B.grad_


In [17]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32).to(device)
target = torch.tensor(dataset['target']).to(device)

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {target.shape}')
print(target)

# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')

# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, n_features = X_train.shape
hidden_units = 30
output_units = 10


shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.], device='cuda:0')
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8], device='cuda:0')
shape of y_true: torch.Size([1797, 10])
shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 10])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 10])


In [21]:
# MLPクラスのインスタンス生成
model = MlpModel(n_features=n_features, hidden_units=hidden_units, output_units=output_units)

# ミニバッチのサイズ定義
batch_size = 32

# 全ミニバッチの数。ミニバッチサイズで割ったときの余りも考慮してプラス１
batch_num = len(y_train) // batch_size + 1

# 各epochごとの学習データ・検証データでの損失記録用
loss_per_epoch_train = []
loss_per_epoch_val = []

# 各epochごとの検証データでのAccuracy結果格納用
accuracy_log = {}


# for文で学習ループ作成
for epoch in range(30):
    # epochごとの損失を蓄積する用の変数
    running_loss = 0
    running_loss_val = 0

    # バッチごとの処理対象データ開始・終了インデックスを初期化
    batch_start_idx = 0
    batch_end_idx = batch_size

    # シャッフル後のindex
    shuffled_indices = np.random.permutation(len(y_train))

    # ミニバッチ勾配降下法
    for i in range(batch_num):
        #シャッフル後のindexからy,Xで同じ範囲を取り出しだしてミニバッチ作成
        batch_indices = shuffled_indices[batch_start_idx : batch_end_idx]
        y_train_batch = y_train[batch_indices, :]
        X_batch = X_train[batch_indices, :]

        # 順伝播の計算
        loss, y_pred = model.forward(X_batch, y_train_batch)
        # 逆伝播の計算
        model.backward(y_train_batch)
        # パラメタの更新
        model.step(learning_rate=learning_rate)
        # 勾配を初期化
        model.zero_grad()

        # 学習データに対するlossの計算・記録
        loss_log.append(loss.item())
        running_loss += loss.item()

        # batch開始・終了インデックスを更新。スライシングの仕様上、endがlen(y_train)を超えても問題ない。
        batch_start_idx += batch_size
        batch_end_idx += batch_size


    # 検証データに対する予測、lossの計算・記録（1epochにつき1回算出）
    loss_val, y_pred_val = model.forward(X_val, y_val)

    # epochの最終的な損失を出力。各バッチの損失の累積を全バッチ数で割って平均を求める
    loss_per_epoch_train.append(running_loss / batch_num)
    loss_per_epoch_val.append(loss_val.item())

    # 検証データに対するaccuracyの計算
    accuracy_log[epoch] = ( (torch.argmax(y_pred_val, dim=1) == y_val.argmax(dim=1)).sum() / len(y_val) ).item()
    print(f'epoch: {epoch}: train error: {running_loss/batch_num}, validation error: {loss_val.item()}, validation accuracy: {accuracy_log[epoch]}')

epoch: 0: train error: 1.8746029138565063, validation error: 1.4139400720596313, validation accuracy: 0.5861111283302307
epoch: 1: train error: 1.1241812189420064, validation error: 0.8558472990989685, validation accuracy: 0.8361111283302307
epoch: 2: train error: 0.7318691783481174, validation error: 0.5895389318466187, validation accuracy: 0.8777778148651123
epoch: 3: train error: 0.5324937442938487, validation error: 0.44710439443588257, validation accuracy: 0.8916667103767395
epoch: 4: train error: 0.419802502128813, validation error: 0.3682768940925598, validation accuracy: 0.9055556058883667
epoch: 5: train error: 0.34980526632732817, validation error: 0.3144545257091522, validation accuracy: 0.925000011920929
epoch: 6: train error: 0.3068399126331011, validation error: 0.27759966254234314, validation accuracy: 0.9277778267860413
epoch: 7: train error: 0.2691398332516352, validation error: 0.2512103319168091, validation accuracy: 0.925000011920929
epoch: 8: train error: 0.2441841